# Update Countries

In [1]:
from dotenv import load_dotenv
load_dotenv()
import ipywidgets as widgets
from IPython.display import display


In [ ]:
from giga.app.config import get_registered_countries

option = widgets.RadioButtons(
    options=['Update an existing country', 'Add a new country'],
    description='Country:',
    disabled=False,
    index=None
)

country_select = widgets.Dropdown(
    options=get_registered_countries(),
    description='Country:',
)

country_input = widgets.Text(
    description='Country:',
)

output = widgets.Output()

# Create a function to handle changes in the selected option
def on_change(change):
    output.clear_output()
    if change['new'] == 'Update an existing country':
        with output:
            display(country_select)
    else:
        with output:
            display(country_input)

# Set the function to be called when the selected option changes
option.observe(on_change, 'value')

# Display the option
display(option, output)

def creating_new_country():
    return option.value == 'Add a new country'

def get_selected_country():
    if not creating_new_country():
        if country_select.value is not None:
            return country_select.value
    elif country_input.value.strip() != "":
        return country_input.value.strip().replace(" ", "_").lower()
    return None


In [ ]:
import json

from ipywidgets import Output, HBox

from giga.utils.globals import COUNTRY_DEFAULT_RELATIVE_DIR
from giga.data.space.country_updater import CountryUpdateRequest
from giga.viz.notebooks.components.widgets.giga_file_upload import GigaFileUpload
from giga.viz.notebooks.components.html.sections import section
from giga.viz.notebooks.cost_estimation_parameter_input import CostEstimationParameterInput

display("Files required for a new country:")

def up_btn(description, accept=".csv", **kwargs):
    layout = widgets.Layout(width='300px')
    return widgets.FileUpload(accept=accept, multiple=False, 
                              description=description, layout=layout, 
                              max_file_size=100,  # MB
                              **kwargs)

# Setting up country defaults

defaults_upload_output = Output()
upload_defaults = GigaFileUpload(accept='.json', multiple=True, description="Import Config")
default_params = CostEstimationParameterInput(local_data_workspace='workspace', show_map=False)
def on_upload_change(change):
    try:
        imported = json.loads(upload_defaults.value[0].content.tobytes())
    except IndexError:
        imported = {}
    defaults_upload_output.clear_output()
    default_params.update(imported)
upload_defaults.observe(on_upload_change, names='value')

country_defaults = up_btn(accept='.csv', description="Country defaults (.csv)")
display(country_defaults)


# Optional info

display("Optional files:")

cellular = up_btn(accept='.csv', description="Cell tower locations (.csv)")
display(cellular)

fiber = up_btn(accept='.csv', description="Fiber node locations (.csv)")
display(fiber)

schools_supplemental = up_btn(accept='.csv', description="Supplemental school info (.csv)")
display(schools_supplemental)

upload_output = widgets.Output()
upload_button = widgets.Button(description="Upload")
display(upload_button, upload_output)

def on_upload_change(change):
    upload_output.clear_output()
    with upload_output:
        req = CountryUpdateRequest()
        req.country_name = get_selected_country()
        req.country_defaults = country_defaults
        req.cellular = cellular
        req.fiber = fiber
        req.schools_supplemental = schools_supplemental
        req.attempt()
        

upload_button.on_click(on_upload_change)
